In [1]:
# get access to the complete repo
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(),"..",".."))
main_copy_dir = os.path.join(parent_dir, "main_copy")
octagon_analysis = os.path.join(parent_dir, "octagon_analysis")
sys.path.append(main_copy_dir)
sys.path.append(octagon_analysis)

print("Added paths to sys.path", main_copy_dir, octagon_analysis)

Added paths to sys.path /Users/benny/Desktop/MSc/Project/Git/repos/main_copy /Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis


In [2]:
# imports
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import analysis.opponent_visibility as opponent_visibility
import trajectory_analysis.trajectory_vectors as trajectory_vectors
import trajectory_analysis.trajectory_headangle as trajectory_headangle
import trajectory_analysis.trajectory_direction as trajectory_direction
import data_extraction.trial_list_filters as trial_list_filters
import data_extraction.get_indices as get_indices
from plotting import plot_octagon, plot_trajectory
import utils.cosine_similarity as cosine_similarity
import data_extraction.extract_trial as extract_trial
import parse_data.identify_filepaths as identify_filepaths
import analysis.loser_inferred_choice as loser_inferred_choice
import plotting.flipped_rotated_trajectory_testing_functions as flipped_rotated_trajectory_testing_functions


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# data folder and filenames
data_folder = "/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files"

social_files, ordered_solos = identify_filepaths.get_filenames(data_folder=data_folder)

In [4]:
# single json filename

json_filename = social_files[17]

In [5]:
_, trial_list = prepare_data.prepare_data(data_folder, json_filename, combine=True)

filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241203_1/2024-12-03_14-31-51_PO03_NN03_Social.json
Loading complete.
Preprocessing complete.


In [31]:
wall_visible = trajectory_headangle.get_wall_visible(trial_list=trial_list, trial_index=17, trial=None, player_id=1, current_fov=110.36, debug=False)

In [28]:
def wall_visibility_player_window(wall_visible, trial, window=5):
    ''' Identify whether either of the relevant walls for this trial are visible between trial start and end time point
        Takes a boolean array of shape num_walls*timepoints which is True when a wall falls within the FoV
        range of the player
        Also takes the trial
        Returns 2 bools, reflecting wall visibility for wall 1 and wall 2 at trial start'''

    # local variables for logic
    wall1_visible = False
    wall2_visible = False
  
    # identify walls
    walls = get_indices.get_walls(trial=trial)
    # take the wall index instead of the wall number, to index wall_visible
    wall1_index = walls[0] - 1
    wall2_index = walls[1] - 1

    # identify which walls are initially visible
    if np.any(wall_visible[wall1_index,0:window]):
        wall1_visible = True
    if np.any(wall_visible[wall2_index,0:window]):
        wall2_visible = True


    return wall1_visible, wall2_visible

In [33]:
trial_index = 17

In [34]:
wall1_visible, wall2_visible = wall_visibility_player_window(wall_visible, trial=trial_list[trial_index], window=5)